In [1]:

import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Transfer Learning
Transfer or inductive learning is a supervised learning technique that reuses parts of a previously trained model on a new network tasked for a different but similar problem. In computer vision, for example, some feature extractors from a nudity detection model could be used to speed up the learning process for a new facial recognition model.

# How is Transfer Learning Used?
Using a pre-trained model significantly reduces the time required for feature engineering and training. The first step is to select a source model, ideally one with a large dataset to train with. Many research institutions release these models and datasets as open-sourced projects, so it’s not necessary to create your own. The next step is to decide which layers to reuse in your own network. The goal is create a framework that is at least better than a naïve model, so you can be assured some new feature learning takes place. Typically deeper layers are reused as these tend to be more general whereas the top layers tend to more finely tuned to a particular problem.
![](
https://www.aismartz.com/blog/wp-content/uploads/2019/11/Concept-of-Transfer-learning.jpg)

At this project , I'm going to use VGG19.Let's look at the VGG19

# What is the VGG19 ?
VGG-19 is a trained Convolutional Neural Network, from Visual Geometry Group, Department of Engineering Science, University of Oxford. The number 19 stands for the number of layers with trainable weights. 16 Convolutional layers and 3 Fully Connected layers.

![](https://qph.fs.quoracdn.net/main-qimg-a1965831bd123c677eeb6cb22bb85bd4)

![](https://miro.medium.com/max/700/1*_Lg1i7wv1pLpzp2F4MLrvw.png)

# LIBRARIES

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.vgg19 import VGG19
import matplotlib.pyplot as plt
from glob import glob

# Read the Dataset

In [1]:
train_path = "/kaggle/input/cell-images-parasitized-or-not/cell_images/train/"
test_path = "/kaggle/input/cell-images-parasitized-or-not/cell_images/test/"

# Exploratory Data Analysis - 1

In [1]:
img = load_img(train_path + "parasitized/C68P29N_ThinF_IMG_20150819_134326_cell_172.png")
plt.imshow(img)
plt.axis("off")
plt.show()

# Exploratory Data Analysis - 2

In [1]:
x = img_to_array(img) # I have been converted image to matrix

print(x.shape)

numberOfClass = len(glob(train_path+"/*"))
print("Number of Class: ",numberOfClass)

# VGG19 MODEL (Transfer Learning CNN STRUCTURE)

In [1]:
vgg = VGG19()


print(vgg.summary())


In [1]:
print(type(vgg))


vgg_layer_list = vgg.layers
print(vgg_layer_list)

model = Sequential()
for i in range(len(vgg_layer_list)-1):
    model.add(vgg_layer_list[i])


print(model.summary())


# What is the Softmax ?
The softmax function is a function that turns a vector of K real values into a vector of K real values that sum to 1. The input values can be positive, negative, zero, or greater than one, but the softmax transforms them into values between 0 and 1, so that they can be interpreted as probabilities. If one of the inputs is small or negative, the softmax turns it into a small probability, and if an input is large, then it turns it into a large probability, but it will always remain between 0 and 1.

The softmax function is sometimes called the softargmax function, or multi-class logistic regression. This is because the softmax is a generalization of logistic regression that can be used for multi-class classification, and its formula is very similar to the sigmoid function which is used for logistic regression. The softmax function can be used in a classifier only when the classes are mutually exclusive.

Many multi-layer neural networks end in a penultimate layer which outputs real-valued scores that are not conveniently scaled and which may be difficult to work with. Here the softmax is very useful because it converts the scores to a normalized probability distribution, which can be displayed to a user or used as input to other systems. For this reason it is usual to append a softmax function as the final layer of the neural network.

![](https://lh6.googleusercontent.com/3vcfJ5hJhsMZAMFIbQOEycfVW1t6rh1CXt62DeMk8RPPXVzV4vCcURNm_z_F7618uAeSHT7qT7wE_UiK5Ic0b-Eeuunn6iTGeHWbpAaUAP6-G2ePubeGWCb4_TmSapeaimZqvuUs)

# RMSPROP (OPTIMIZER)

RMSprop is a gradient-based optimization technique used in training neural networks. It was proposed by the father of back-propagation, Geoffrey Hinton. Gradients of very complex functions like neural networks have a tendency to either vanish or explode as the data propagates through the function (refer to vanishing gradients problem). Rmsprop was developed as a stochastic technique for mini-batch learning. RMSprop deals with the above issue by using a moving average of squared gradients to normalize the gradient. This normalization balances the step size (momentum), decreasing the step for large gradients to avoid exploding and increasing the step for small gradients to avoid vanishing. Simply put, RMSprop uses an adaptive learning rate instead of treating the learning rate as a hyperparameter. This means that the learning rate changes over time.

![](https://miro.medium.com/max/1174/1*bdgqAQdEgpyBZscgNgx6hQ.png)


In [1]:
for layers in model.layers:
    layers.trainable = False
    
model.add(Dense(numberOfClass,activation="softmax"))

print(model.summary())


model.compile(loss = "categorical_crossentropy",
              optimizer = "rmsprop",
              metrics = ["accuracy"]
              )

# Training the Model

In [1]:
train_data = ImageDataGenerator().flow_from_directory(train_path,target_size = (224,224))
test_data = ImageDataGenerator().flow_from_directory(test_path,target_size = (224,224))

batch_size = 32

hist = model.fit_generator(train_data,
                           steps_per_epoch=3200//batch_size,
                           epochs = 100,
                           validation_data=test_data,
                           validation_steps=800//batch_size
                           )

In [1]:
print(hist.history.keys())
plt.figure(figsize=(15,15))
plt.plot(hist.history["loss"],label = "training loss")
plt.plot(hist.history["val_loss"],label = "validation loss")
plt.title("Training Loss vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(15,15))
plt.plot(hist.history["accuracy"],label = "training acc")
plt.plot(hist.history["val_accuracy"],label = "validation acc")
plt.title("Training Accuracy vs Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Accuracy Calculation

In [1]:
results = model.evaluate(test_data)